# Factorization into q-integers

In many cases we have a nice product formula and its q-analog.
For example, the number of standard Young tableaux of shape $\lambda\vdash n$ is
$$
f^\lambda = \frac{n!}{\prod_{u\in \lambda} h(u)},
$$
where $h(u)$ is the hook length of the cell $u$ in the Young diagram $\lambda$. Its $q$-analog is
$$
\sum_{T\in SYT(\lambda)} q^{maj(T)} = \frac{[n]_q!}{\prod_{u\in \lambda} [h(u)]_q},
$$


In this Sage Notebook we factor a rational function in $q$ into $q$-integers $[k]_q = q^k-1$. (Note that we use a rather different convention for $q$-integers.)

In the code we use cyclotomic polynomials.
Recall that the cyclotomic polynomials $\Phi_n(x)$ satisfy
$$
x^N - 1 = \prod_{d|N} \Phi_d(x).
$$

In [118]:
max_cyclotomic_index = 100   
var('q','x','y','z')
var('a','b','c','d','e','k','m')

def find_cyclotomic_index(p,x,N=max_cyclotomic_index):
    r"""
    Return n if p is the nth cyclotomic polynomial for some 1<=n<=N and 0 otherwise.
        
    EXAMPLES::

        sage: find_cyclotomic_index(1+q^4,q)
        8
        sage: find_cyclotomic_index(1-q,q)
        0
        sage: find_cyclotomic_index(q-1,q)
        1

    """
    for i in range(1,N+1):
        if p == cyclotomic_polynomial(i,x):
            return i
    return 0

def find_qint(f,q):
    """
    Return x if f = q^x-1 and 0 otherwise
    """
    if f == q-1:
        return 1
    if (f+1).operator() == (x^y).operator():
        A = (f+1).operands()
        if A[0] == q:
            return A[1]
    return 0

def qint_expression(f,q,N=max_cyclotomic_index):
    r"""
    Return [const,qnum,qden] where if F = C*NUM/DEN, qnum and qden are the lists of factors in NUM and DEN which are products of factors of the form (1-q^k).
        
    EXAMPLES::

        sage: qint_expression(-q^2*(1-q)*(1-q^6)^2/(1-q^4)/(1-q^18),q)
        [q^2, [6, 6, 1], [18, 4]]
        sage: qint_expression(-q^2*(1+q)^2/(1-q^2+q^4),q)
        [-q^2, [6, 4, 2], [12]]
        sage: qint_expression(-q^2*(1+q)^2*(1-q^6)/(1-q^4),q)
        [-q^2, [6, 2, 2], [4]]

    """
    const,qnum,qden,num,den = 1,[],[],[],[]
    for A in f.factor_list():
        k = find_qint(A[0],q)
        if k != 0:
            if A[1] > 0:
                qnum += [k for i in range(A[1])]
            else:
                qden += [k for i in range(-A[1])]
        else:
            n = find_cyclotomic_index(A[0],q,N)
            if n == 0:
                const *= A[0]^A[1]
            elif A[1] > 0:
                num += [n for i in range(A[1])]
            else:
                den += [n for i in range(-A[1])]
    while len(num)+len(den) > 0:
        num.sort()
        den.sort()
        if num == []:
            k = den.pop()
            qden.append(k)
            num += divisors(k)[:-1]
        elif den == []:
            k = num.pop()
            qnum.append(k)
            den += divisors(k)[:-1]
        elif num[-1] == den[-1]:
            num.pop()
            den.pop()
        elif num[-1] < den[-1]:
            k = den.pop()
            qden.append(k)
            num += divisors(k)[:-1]
        elif num[-1] > den[-1]:
            k = num.pop()
            qnum.append(k)
            den += divisors(k)[:-1]
    while 1 in qden and 1 in qnum:
        qden.remove(1)
        qnum.remove(1)
    qden.sort(reverse=True)
    qnum.sort(reverse=True)
    return [const,qnum,qden]

In [121]:
var('q')
A=add(q^(T.standard_major_index()) for T in StandardTableaux([4,3,1]))
factor(A)

(q^6 + q^5 + q^4 + q^3 + q^2 + q + 1)*(q^4 + q^3 + q^2 + q + 1)*(q^4 + 1)*q^5

In the above example, it is not easy to see directly the q-integer factors. More obvious example would be $[n]_q!$.

In [95]:
B = factor(mul(q^k-1 for k in range(1,10))); B

(q^6 + q^5 + q^4 + q^3 + q^2 + q + 1)*(q^6 + q^3 + 1)*(q^4 + q^3 + q^2 + q + 1)*(q^4 + 1)*(q^2 + q + 1)^3*(q^2 - q + 1)*(q^2 + 1)^2*(q + 1)^4*(q - 1)^9

If we used "qint_expression" method we can see the q-integer factors.

In [122]:
qint_expression(A,q)

[q^5, [8, 7, 5], [4, 1, 1]]

The above output means that
$$
A = q^5 \frac{[8]_q[7]_q[5]_q}{[4]_q[1]_q[1]_q}.
$$

In [124]:
qint_expression(B,q)

[1, [9, 8, 7, 6, 5, 4, 3, 2, 1], []]